<a href="https://colab.research.google.com/github/mkpvasu/Brain-Tumor-Classification/blob/main/BT_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from torchvision.utils import make_grid
import os
import random
import numpy as np
import pandas as pd
import pickle
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from google.colab import drive 

In [2]:
from google.colab import drive
drive.mount('/content/Drive')

Mounted at /content/Drive


In [3]:
torch.__version__

'1.11.0+cu113'

In [4]:
!nvidia-smi

Sat May 28 20:04:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
torch.cuda.empty_cache()

In [37]:
class BrainTumorDataset(Dataset):
  def __init__(self, images, labels):
    # Images
    self.X = images
    # Corresponding Labels
    self.y = labels
    
    # Convert original image numpy array to PIL image and then to a tensor
    self.transform = transforms.Compose([transforms.ToPILImage(),
        #transforms.Resize((64,64)),
        transforms.ToTensor()
    ])

    # Augmenting original image numpy array by resizing it to (64,64) pixels, then rotating it to randomly between [-k,k] degrees and 
    # then convert it to tensor
    self.transform30 = transforms.Compose([
        transforms.ToPILImage(),
        #transforms.Resize((64,64)),                                         
        transforms.RandomRotation(30),
        transforms.ToTensor()                                 
    ])

    self.transform90 = transforms.Compose([
        transforms.ToPILImage(),
        #transforms.Resize((64,64)),
        transforms.RandomRotation(90),
        transforms.ToTensor()                                  
    ])

    self.transform135 = transforms.Compose([
        transforms.ToPILImage(),
        #transforms.Resize((64,64)),
        transforms.RandomRotation(135),
        transforms.ToTensor()                                  
    ])

    self.transform180 = transforms.Compose([
        transforms.ToPILImage(),
        #transforms.Resize((64,64)),
        transforms.RandomRotation(180),
        transforms.ToTensor()                                
    ])

    self.transform270 = transforms.Compose([
        transforms.ToPILImage(),
        #transforms.Resize((64,64)),
        transforms.RandomRotation(270),
        transforms.ToTensor()                                
    ])

    self.transform315 = transforms.Compose([
        transforms.ToPILImage(),
        #transforms.Resize((64,64)),
        transforms.RandomRotation(315),
        transforms.ToTensor()                                
    ])
    
    self.transform345 = transforms.Compose([
        transforms.ToPILImage(),
        #transforms.Resize((64,64)),
        transforms.RandomRotation(345),
        transforms.ToTensor()                                
    ])

  def __len__(self):
    # Returns # of images
    return len(self.X)

  def __getitem__(self, idx):
    # Transformations for one image of X at a time
    # Original image as a tensor
    img = self.transform(self.X[idx])

    # Transformations are called to augment images and save it to traing the model
    img30 = self.transform30(self.X[idx])
    img90 = self.transform90(self.X[idx])
    img135 = self.transform135(self.X[idx])
    img180 = self.transform180(self.X[idx])
    img270 = self.transform270(self.X[idx])
    img315 = self.transform270(self.X[idx])  
    img345 = self.transform270(self.X[idx])    
    
    # store the transformed images in a list
    batch = [img, img30, img90, img135, img180, img270, img315, img345]

    # One Hot Encoding
    labels = torch.zeros(4, dtype=torch.float32)
    labels[int(self.y[idx])] = 1.0

    b_labels = [labels, labels, labels, labels, labels, labels, labels, labels]

    # 8 augmented images and corresponding labels per sample will be returned
    return (torch.stack(b_labels), torch.stack(batch))

In [7]:
train_data = pickle.load(open('/content/Drive/My Drive/Colab Notebooks/braintumordata/train_data.pickle', 'rb'))

In [8]:
Xt, yt = [], []
features, labels = None, None
label = []

In [9]:
for features,labels in train_data:
  Xt.append(features)
  yt.append(labels)

In [10]:
# The train_data has been divided into training, validation and testing data
# 75% - training data
# 15% - validation data
# 10% - testing data

X_train, X_test, y_train, y_test = train_test_split(Xt, yt, test_size=0.25, shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.6, shuffle=True)

In [11]:
# Clearing memory
Xt, yt, labels, label, features, train_data = None, None, None, None, None, None

In [28]:
class BrainTumorDataset(Dataset):
  def __init__(self, images, labels):
    # images
    self.X = images
    # labels
    self.y = labels
    
    # Transformation for converting original image array to an image and then convert it to a tensor
    self.transform = transforms.Compose([transforms.ToPILImage(),
                                         transforms.Resize((64,64)),
        transforms.ToTensor()
    ])

    # Transformation for converting original image array to an image, rotate it randomly between -45 degrees and 45 degrees, and then convert it to a tensor
    self.transform1 = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((64,64)),                                          
        transforms.RandomRotation(45),
        transforms.ToTensor()                                 
    ])

    # Transformation for converting original image array to an image, rotate it randomly between -90 degrees and 90 degrees, and then convert it to a tensor
    self.transform2 = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((64,64)),
        transforms.RandomRotation(90),
        transforms.ToTensor()                                  
    ])

    # Transformation for converting original image array to an image, rotate it randomly between -120 degrees and 120 degrees, and then convert it to a tensor
    self.transform3 = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((64,64)),
        transforms.RandomRotation(120),
        transforms.ToTensor()                                  
    ])

    # Transformation for converting original image array to an image, rotate it randomly between -180 degrees and 180 degrees, and then convert it to a tensor
    self.transform4 = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((64,64)),
        transforms.RandomRotation(180),
        transforms.ToTensor()                                
    ])

    # Transformation for converting original image array to an image, rotate it randomly between -270 degrees and 270 degrees, and then convert it to a tensor
    self.transform5 = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((64,64)),
        transforms.RandomRotation(270),
        transforms.ToTensor()                                
    ])

    # Transformation for converting original image array to an image, rotate it randomly between -300 degrees and 300 degrees, and then convert it to a tensor
    self.transform6 = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((64,64)),
        transforms.RandomRotation(300),
        transforms.ToTensor()                               
    ])

    # Transformation for converting original image array to an image, rotate it randomly between -330 degrees and 330 degrees, and then convert it to a tensor
    self.transform7 = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((64,64)),
        transforms.RandomRotation(330),
        transforms.ToTensor()                                 
    ])

  def __len__(self):
    # return length of image samples
    return len(self.X)

  def __getitem__(self, idx):
    # perform transformations on one instance of X
    # Original image as a tensor
    data = self.transform(self.X[idx])

    # Augmented image at 45 degrees as a tensor
    aug45 = self.transform1(self.X[idx])

    # Augmented image at 90 degrees as a tensor
    aug90 = self.transform2(self.X[idx])

    # Augmented image at 120 degrees as a tensor
    aug120 = self.transform3(self.X[idx])

    # Augmented image at 180 degrees as a tensor
    aug180 = self.transform4(self.X[idx])

    # Augmented image at 270 degrees as a tensor
    aug270 = self.transform5(self.X[idx])

    # Augmented image at 300 degrees as a tensor
    aug300 = self.transform6(self.X[idx])

    # Augmented image at 330 degrees as a tensor
    aug330 = self.transform7(self.X[idx])      
    
    # store the transformed images in a list
    new_batch = [data, aug45, aug90, aug120, aug180, aug270, aug300, aug330]

    # one-hot encode the labels
    labels = torch.zeros(4, dtype=torch.float32)
    labels[int(self.y[idx])] = 1.0

    new_labels = [labels, labels, labels, labels, labels, labels, labels, labels]

    # 8 augmented images and corresponding labels per sample will be returned
    return (torch.stack(new_labels), torch.stack(new_batch))

In [38]:
# To augmented dataset and save it to corresponding variables

train_set = BrainTumorDataset(X_train, y_train)
val_set = BrainTumorDataset(X_val, y_val)
test_set = BrainTumorDataset(X_test, y_test)

In [39]:
# These dataloaders load the dataset to minimize the time device has to wait to retrieve data

train_loader = DataLoader(train_set, batch_size=4, shuffle=True, pin_memory=True, num_workers=8)
val_loader = DataLoader(val_set, batch_size=4, shuffle=True, pin_memory=True, num_workers=8)
test_loader = DataLoader(test_set, batch_size=10, shuffle=True, pin_memory=True, num_workers=8)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [40]:
device_name = "cuda:0" if torch.cuda.is_available() else "cpu"
device = torch.device(device_name)

In [ ]:
# Using Resnet50 (50 layers) pretrained model (this model will be pretrined using ImageNet dataset) as our model
resnet_50 = models.resnet50(pretrained=True)

# Training all parameters
for param in resnet_50.parameters():
    param.requires_grad = True

# Input for fully connected layer
n_inputs = resnet_50.fc.in_features

# Creating a new final fully connected layer to make this model perform classification for our dataset (Original pretrined model can classify the 
# classes it was trained on)
resnet_50.fc = nn.Sequential(nn.Linear(n_inputs, 2048),
                                nn.SELU(),
                                nn.Dropout(p=0.4),
                                nn.Linear(2048, 2048),
                                nn.SELU(),
                                nn.Dropout(p=0.4),
                                nn.Linear(2048, 4),
                                nn.LogSigmoid())

# set all paramters of the model as trainable
for name, child in resnet_50.named_children():
  for name2, params in child.named_parameters():
    params.requires_grad = True

# set model to run on GPU or CPU absed on availibility
resnet_50.to(device)

In [42]:
# Cross Entropy loss is used for classification algorithm here as it is common
# GPU is used if available for faster training
crossen_loss = nn.CrossEntropyLoss().to(device)

# Stochastic Gradient Descent optimizer
sgd_opt = torch.optim.SGD(resnet_50.parameters(), momentum=0.9, lr=3e-4)

# # of epochs to be trained
epochs = 30

# Lists to store train, test losses and accuracies
train_losses, test_losses, train_accs, test_accs = [],[],[],[]

In [43]:
def save_model(state, is_best, filename='/content/Drive/My Drive/Colab Notebooks/braintumordata/bt_resnet50_bt_intermodel.pth.tar'):
    torch.save(state, filename)

In [44]:
torch.cuda.empty_cache()

In [ ]:
# set training start time
start_time = time.time()

# set best_prec loss value as 2 for checkpoint threshold
best_loss = 2.5

# empty batch variables
b = None
train_b = None
test_b = None

# start training
for i in range(epochs):
    # empty training correct and test correct counter as 0 during every iteration
    trn_corr = 0
    tst_corr = 0
    
    # set epoch's starting time
    e_start = time.time()
    
    # train in batches
    for b, (y, X) in enumerate(train_loader):
        # set label as cuda if device is cuda
        X, y = X.to(device), y.to(device)

        # forward pass image sample
        y_pred = resnet_50(X.view(-1, 3, 512, 512))
        # calculate loss
        loss = crossen_loss(y_pred.float(), torch.argmax(y.view(32, 4), dim=1).long())

        # get argmax of predicted tensor, which is our label
        predicted = torch.argmax(y_pred, dim=1).data
        # if predicted label is correct as true label, calculate the sum for samples
        batch_corr = (predicted == torch.argmax(y.view(32, 4), dim=1)).sum()
        # increment train correct with correcly predicted labels per batch
        trn_corr += batch_corr
        
        # set optimizer gradients to zero
        sgd_opt.zero_grad()
        # back propagate with loss
        loss.backward()
        # perform optimizer step
        sgd_opt.step()

    # set epoch's end time
    e_end = time.time()
        # print training metrics
    print(f'Epoch {(i+1)} Batch {(b+1)*4}\nAccuracy: {trn_corr.item()*100/(4*8*b):2.2f} %  Loss: {loss.item():2.4f}  Duration: {((e_end-e_start)/60):.2f} minutes') # 4 images per batch * 8 augmentations per image * batch length

    # some metrics storage for visualization
    train_b = b
    train_losses.append(loss)
    train_accs.append(trn_corr)

    X, y = None, None

    # validate using validation generator
    # do not perform any gradient updates while validation
    with torch.no_grad():
        for b, (y, X) in enumerate(val_loader):
            # set label as cuda if device is cuda
            X, y = X.to(device), y.to(device)

            # forward pass image
            y_val = resnet_50(X.view(-1, 3, 512, 512))

            # get argmax of predicted tensor, which is our label
            predicted = torch.argmax(y_val, dim=1).data

            # increment test correct with correcly predicted labels per batch
            tst_corr += (predicted == torch.argmax(y.view(32, 4), dim=1)).sum()

    # get loss of validation set
    loss = crossen_loss(y_val.float(), torch.argmax(y.view(32, 4), dim=1).long())
    # print validation metrics
    print(f'Validation Accuracy {tst_corr.item()*100/(4*8*b):2.2f} Validation Loss: {loss.item():2.4f}\n')

    # if current validation loss is less than previous iteration's validatin loss create and save a checkpoint
    is_best = loss < best_loss
    best_loss = min(loss, best_loss)
    save_model({
            'epoch': i + 1,
            'state_dict': resnet_50.state_dict(),
            'best_prec1': best_loss,
        }, is_best)

    # some metrics storage for visualization
    test_b  = b
    test_losses.append(loss)
    test_accs.append(tst_corr)

# set total training's end time
end_time = time.time() - start_time    

# print training summary
print("\nTraining Duration {:.2f} minutes".format(end_time/60))
print("GPU memory used : {} kb".format(torch.cuda.memory_allocated()))
print("GPU memory cached : {} kb".format(torch.cuda.memory_cached()))

In [ ]:
torch.cuda.empty_cache()

In [ ]:
torch.save(resnet_model.state_dict(), '/content/drive/My Drive/bt_resnet50_model.pt')

In [ ]:
print(f'Validation accuracy: {test_correct[-1].item()*100/(test_b*8*4):.2f}%')

In [ ]:
plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='Validation loss')
plt.title('Loss Metrics')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend()
plt.show()

In [ ]:
plt.plot([t/171 for t in train_correct], label='Training accuracy')
plt.plot([t/36 for t in test_correct], label='Validation accuracy')
plt.title('Accuracy Metrics')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend()
plt.show()

In [ ]:
train_gen = None
valid_gen = None
train_set = None
valid_set = None

In [ ]:
# set model to evaluation mode
resnet_model.eval()

# perform no gradient updates
with torch.no_grad():
    # soem metrics storage for visualization and analysis
    correct = 0
    test_loss = []
    test_corr = []
    labels = []
    pred = []
    # perform test set evaluation batch wise
    for (y, X) in test_gen:
        # set label to use CUDA if available
        X, y = X.to(device), y.to(device)

        # append original labels
        labels.append(torch.argmax(y.view(10 * 8, 4), dim=1).data)

        # perform forward pass
        y_val = resnet_model(X.view(-1, 3, 512, 512))

        # get argmax of predicted values, which is our label
        predicted = torch.argmax(y_val, dim=1).data
        # append predicted label
        pred.append(predicted)

        # calculate loss
        loss = criterion(y_val.float(), torch.argmax(y.view(10 * 8, 4), dim=1).long())

        # increment correct with correcly predicted labels per batch
        correct += (predicted == torch.argmax(y.view(10 * 8, 4), dim=1)).sum()

        # append correct samples labels and losses
        test_corr.append(correct)
        test_loss.append(loss)
        
print(f"Test Loss: {test_loss[-1].item():.4f}")

In [ ]:
print(f'Test accuracy: {test_corr[-1].item()*100/(460*8):.2f}%')

In [ ]:
labels = torch.stack(labels)
pred = torch.stack(pred)

In [ ]:
LABELS = ['Meningioma', 'Glioma', 'Pitutary']

In [ ]:
arr = confusion_matrix(pred.view(-1).cpu(), labels.view(-1).cpu())
df_cm = pd.DataFrame(arr, LABELS, LABELS)
plt.figure(figsize = (9,6))
sns.heatmap(df_cm, annot=True, fmt="d", cmap='viridis')
plt.xlabel("Prediction")
plt.ylabel("Target")
plt.show()

In [ ]:
print(f"Clasification Report\n\n{classification_report(pred.view(-1).cpu(), labels.view(-1).cpu())}")